# XXX - topics & named-entities

In [1]:
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from nltk import word_tokenize, corpus
import gensim
from gensim import corpora,models
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords
import pyLDAvis.gensim_models
from pprint import pprint

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-df-2.csv', \
                 encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\charliehebdo-src-2.csv', \
                        encoding = 'utf-8', header =0)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
len(src_tw_df.loc[src_tw_df.label==0])+len(df.loc[df.label==0]) #number of rumorous tweets

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


28978

In [4]:
rumor_tw = df.cleaned_reply_tw[df.label ==0 ].append(src_tw_df.cleaned_src_tw[src_tw_df.label ==0])

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
non_rumor_tw = df.cleaned_reply_tw[df.label ==1 ].append(src_tw_df.cleaned_src_tw[src_tw_df.label ==1])

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
rumor_tw_list = rumor_tw.astype(str).apply(word_tokenize).tolist()
non_rumor_tw_list = non_rumor_tw.astype(str).apply(word_tokenize).tolist()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LDA model

### Rumor

In [7]:
#Rumor dictionary
id2word = corpora.Dictionary(rumor_tw_list)
print(id2word)

Dictionary(23950 unique tokens: ['dead', 'shoot', 'today', 'consider', 'guess']...)


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Term Document Frequency
corpus_r = [id2word.doc2bow(tw) for tw in rumor_tw_list]

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def compute_coherence_values(corpus, dictionary, k):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)
    
    coherence_model_lda = CoherenceModel(model=lda_model, 
                                         texts=rumor_tw_list, #要改
                                         dictionary=id2word,    #要改
                                         coherence='c_v')
    return coherence_model_lda.get_coherence()
    

In [ ]:
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

In [ ]:
model_results ={'Topics': [], 'Coherence': []}

for k in topics_range:

    # get the coherence score for the given parameters
    cv = compute_coherence_values(corpus=corpus_r, 
                                  dictionary=id2word,k=k)
    # Save the model results
    model_results['Topics'].append(k)
    model_results['Coherence'].append(cv)


In [ ]:
model_results = pd.DataFrame(model_results)
model_results

In [ ]:
# Show graph
import matplotlib.pyplot as plt
limit=11; start=2; step=1;
x = range(start, limit, step)
plt.plot(model_results.Topics, model_results.Coherence)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

Pick **k = 4**

In [ ]:
lda_model_r = gensim.models.LdaMulticore(corpus=corpus_r,id2word=id2word,num_topics=4, 
                                       random_state=100,chunksize=100,passes=10)

In [ ]:
rumor_topics = pyLDAvis.gensim_models.prepare(lda_model_r,corpus_r,id2word)

In [ ]:
pyLDAvis.display(rumor_topics)

In [ ]:
p_r = pyLDAvis.gensim_models.prepare(lda_model_r, corpus_r, id2word)

In [ ]:
save_path = 'D:/論文/PHEME9/Code/graph/lda\\'
pyLDAvis.save_html(p_r, save_path+'charliehebdo_lda_r.html')

In [ ]:
# Print the Keyword in the 4 topics
topic_term = pd.DataFrame()

for topic in lda_model_r.print_topics(num_words = 30):
    terms = [] 
#     print(topic[0],":")
    listOfTerms = topic[1].split('+')
    for term in listOfTerms:
        listItems = term.split('*')
        terms.append(listItems[1])
    topic_term.insert(topic[0],"#"+str(topic[0]) ,terms)
topic_term

topic_term.to_csv('D:/論文/PHEME9/Code/charliehebdo/LDA_terms_r_charliehebdo.csv', index=False)

### Non-rumor

In [ ]:
id2word_nr = corpora.Dictionary(non_rumor_tw_list)
corpus_nr = [id2word_nr.doc2bow(tw) for tw in non_rumor_tw_list]

In [ ]:
def compute_coherence_values(corpus, dictionary, k):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10)
    
    coherence_model_lda = CoherenceModel(model=lda_model, 
                                         texts=non_rumor_tw_list, #要改
                                         dictionary=id2word_nr,    #要改
                                         coherence='c_v')
    return coherence_model_lda.get_coherence()
    

In [ ]:
model_results_nr ={'Topics': [], 'Coherence': []}

for k in topics_range:

    # get the coherence score for the given parameters
    cv = compute_coherence_values(corpus=corpus_nr, 
                                  dictionary=id2word_nr,k=k)
    # Save the model results
    model_results_nr['Topics'].append(k)
    model_results_nr['Coherence'].append(cv)


In [ ]:
model_results_nr = pd.DataFrame(model_results_nr)
model_results_nr

In [ ]:
# Show graph
import matplotlib.pyplot as plt
limit=11; start=2; step=1;
x = range(start, limit, step)
plt.plot(model_results_nr.Topics, model_results_nr.Coherence)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

Pick **k = 3**

In [ ]:
lda_model_nr = gensim.models.LdaMulticore(corpus=corpus_nr,id2word=id2word_nr,num_topics=3, 
                                       random_state=100,passes=10)

In [ ]:
non_rumor_topics = pyLDAvis.gensim_models.prepare(lda_model_nr,corpus_nr,id2word_nr)

In [ ]:
pyLDAvis.display(non_rumor_topics)

In [ ]:
pprint(lda_model_nr.print_topics())

In [ ]:
p_nr = pyLDAvis.gensim_models.prepare(lda_model_nr, corpus_nr, id2word_nr)

In [ ]:
save_path = 'D:/論文/PHEME9/Code/graph/lda\\'
pyLDAvis.save_html(p_nr, save_path+'charliehebdo_lda_nr.html')

In [ ]:
# Print the Keyword in the 4 topics
topic_term_nr = pd.DataFrame()

for topic in lda_model_nr.print_topics(num_words = 30):
    terms = [] 
#     print(topic[0],":")
    listOfTerms = topic[1].split('+')
    for term in listOfTerms:
        listItems = term.split('*')
        terms.append(listItems[1])
    topic_term_nr.insert(topic[0],"#"+str(topic[0]) ,terms)
topic_term_nr

topic_term_nr.to_csv('D:/論文/PHEME9/Code/charliehebdo/LDA_terms_nr_charliehebdo.csv', index=False)

## BERTopic modeling

In [9]:
!pip install -v bertopic==0.3.1

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using pip 22.2.2 from c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pip (python 3.9)
  Using cached bertopic-0.3.1-py2.py3-none-any.whl (14 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command python setup.py egg_info
ERROR: No .egg-info directory found in C:\Users\user\AppData\Local\Temp\pip-pip-egg-info-zoqy1ojb


In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
# topics, probs = topic_model.fit_transform(docs)

## Clustering Algorithms

In [10]:
from sklearn.cluster import AgglomerativeClustering, KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Rumor

In [11]:
vectorizer = TfidfVectorizer(max_df=0.7,min_df=0.01,stop_words="english")
X = vectorizer.fit_transform(rumor_tw.astype(str).tolist())
print(X.shape)

(28978, 69)


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Kmeans

In [ ]:
def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1, 1)
    
    sse = []
    for k in iters:
        sse.append(KMeans(n_clusters=k, random_state=100).fit(data).inertia_)
       
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(X, 10)

Pick **no. of cluster = 4**

In [ ]:
clusters = KMeans(n_clusters=4, random_state=100).fit_predict(X)

In [ ]:
pd.DataFrame(X.todense()).groupby(clusters).mean()

In [ ]:
def get_top_keywords(data, clusters, labels, n_terms):
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    cluster_term = pd.DataFrame()
    
    for i,r in df.iterrows():
        terms = []
        for t in np.argsort(r)[-n_terms:]:
            terms.append(labels[t] )
        cluster_term.insert(i,str(i),terms)
    
    return cluster_term

In [ ]:
kmeans_r = get_top_keywords(X, clusters, vectorizer.get_feature_names(), 20)
kmeans_r

In [ ]:
kmeans_r.to_csv('D:/論文/PHEME9/Code/charliehebdo/kmeans_terms_r_charliehebdo.csv', index=False)

#### Hierarchical Clustering

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import dendrogram, linkage, ward

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
model = AgglomerativeClustering(n_clusters=None, distance_threshold = 0.2)
model.fit(X.todense())

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


AgglomerativeClustering(distance_threshold=0.2, n_clusters=None)

In [14]:
model.labels_

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 257, 4919,  817, ..., 2136,  743,  854], dtype=int64)

In [ ]:
dist = 1 - cosine_similarity(X)
linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig = plt.figure(figsize=(5, 3))
dn = dendrogram(linkage_matrix)
plt.show()

### Non-Rumor

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.7,min_df=0.01,stop_words="english")
Y = vectorizer.fit_transform(non_rumor_tw.astype(str).tolist())
print(Y.shape)

#### Kmeans 

In [ ]:
find_optimal_clusters(Y, 10)

Pick **no. of cluster = 4**

In [ ]:
clusters = KMeans(n_clusters=3, random_state=100).fit_predict(Y)

In [ ]:
kmeans_nr = get_top_keywords(Y, clusters, vectorizer.get_feature_names(), 20)
kmeans_nr

In [ ]:
kmeans_nr.to_csv('D:/論文/PHEME9/Code/charliehebdo/kmeans_terms_nr_charliehebdo.csv', index=False)

#### Hierachical Clustering

In [ ]:
model_nr = AgglomerativeClustering(compute_distances = True,n_clusters=3)
model_nr.fit(Y.todense())

In [ ]:
model_nr.labels_

## Named entity recognition. 

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")

### Rumor

In [ ]:
a = rumor_tw.astype('str').apply(nlp)

In [ ]:
a[0]

In [ ]:
entity = []
label = []
tw_ent = []

for i in range(len(a)):
    each_tw_ents = a.iloc[i].ents
    
    for ent in each_tw_ents:
        entity.append(ent.text)
        label.append(ent.label_)
    
    tw_ent.append(list(each_tw_ents))

In [ ]:
entities = pd.DataFrame({'Entity': entity, 'label':label})
entities

In [ ]:
top50_ent_r = entities.Entity.value_counts()[:50,].to_frame().reset_index()
top50_ent_r.columns=['Entity','Count']
top50_ent_r['Percentage'] = top50_ent_r['Count'] / sum(top50_ent_r.Count)
top50_ent_r

In [ ]:
top50_ent_r.to_csv('D:/論文/PHEME9/Code/charliehebdo/top50_ent_r.csv', index=False)

### Non-rumor

In [ ]:
b = non_rumor_tw.astype('str').apply(nlp)

In [ ]:
entity_nr = []
label_nr = []
nr_tw_ent = []

for i in range(len(b)):
    each_tw_ents = b.iloc[i].ents
    
    for ent in each_tw_ents:
        entity_nr.append(ent.text)
        label_nr.append(ent.label_)
    
    nr_tw_ent.append(list(each_tw_ents))

In [ ]:
entities_nr = pd.DataFrame({'Entity': entity_nr, 'label':label_nr})
entities_nr

In [ ]:
top50_ent_nr = entities_nr.Entity.value_counts()[:50,].to_frame().reset_index()
top50_ent_nr.columns=['Entity','Count']
top50_ent_nr['Percentage'] = top50_ent_nr['Count'] / sum(top50_ent_nr.Count)
top50_ent_nr

In [ ]:
top50_ent_nr.to_csv('D:/論文/PHEME9/Code/charliehebdo/top50_ent_nr.csv', index=False)

## POS Tagging

### Rumor

In [ ]:
tagging_r = []

for i in range(len(a)):
    for token in a.iloc[i]:
        tagging_r.append(token.tag_)
# tagging_r     

In [ ]:
from collections import Counter
tag_set_r = Counter(tagging_r)
tag_r = pd.DataFrame.from_dict(tag_set_r, orient='index').reset_index()
tag_r.columns=['tag','count']
tag_r.sort_values(by='count',inplace=True,ascending=False)
tag_r.reset_index(drop=True, inplace=True)
tag_r
tag_r.to_csv('D:/論文/PHEME9/Code/charliehebdo/tagging_r.csv', index=False)

### Non-rumor

In [ ]:
tagging_nr = []
pos_nr = []

for i in range(len(b)):
    for token in b.iloc[i]:
        tagging_nr.append(token.tag_)

# tagging_nr    

In [ ]:
tag_set_nr = Counter(tagging_nr)
tag_nr = pd.DataFrame.from_dict(tag_set_nr, orient='index').reset_index()
tag_nr.columns=['tag','count']
tag_nr.sort_values(by='count',inplace=True,ascending=False)
tag_nr.reset_index(drop=True, inplace=True)
tag_nr
tag_nr.to_csv('D:/論文/PHEME9/Code/charliehebdo/tagging_nr.csv', index=False)

In [ ]:
tag_nr